In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import re
from re import match
import getpass
import io
import os, json
from py2neo import Graph as GRAPHS


In [57]:
gov_source = pd.read_csv (r'C:\Users\VBAPHCColleD\Downloads\source_gov.csv',skip_blank_lines=True) 
gov_source.dropna(how="all", inplace=True)
print(gov_source.columns)

Index(['@timestamp', 'source.ip', 'network.verb', 'destination.port',
       'network.total_bytes', 'network.total_packets', 'network.tcp_flags',
       'organization.name', 'destination.ip', 'destination.geoip.country_name',
       'message', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5',
       'source_report', 'soure.ip.intel', 'destination.ip.intel'],
      dtype='object')


In [58]:
mal_source = pd.read_csv (r'C:\Users\user\Downloads\source_mal.csv',skip_blank_lines=True) 
mal_source.dropna(how="all", inplace=True)
print(mal_source.columns)

Index(['@timestamp', 'source.ip', 'network.verb', 'destination.port',
       'network.total_bytes', 'network.total_packets', 'network.tcp_flags',
       'organization.name', 'destination.ip', 'destination.geoip.country_name',
       'message', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5',
       'source_report', 'soure.ip.intel', 'destination.ip.intel'],
      dtype='object')


In [5]:
graph = GRAPHS("bolt://localhost:7687", user="neo4j", password="temp",name="switches")

In [61]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
tx = graph.begin()
for index, row in mal_source.iterrows():
    tx.evaluate('''
    MERGE (s:external_host {IP: $SourceIP})
    MERGE (d:government_host {IP: $DestinationIP})
    CREATE (s)-[i:session]->(d)
    SET i.Bytes = toInteger($bytes),i.DestinationPort=$DestinationPort,i.Time=$time,i.Packets=$packets,i.soure_ip_intel=$soure_ip_intel,i.destination_ip_intel=$destination_ip_intel
    ''', parameters = {'SourceIP': row['source.ip'], 'DestinationIP': row['destination.ip'],'DestinationPort': row['destination.port'],'packets': row['network.total_packets'],'bytes': row['network.total_bytes'],'time': row['@timestamp'],'soure_ip_intel': row['soure.ip.intel'],'destination_ip_intel': row['destination.ip.intel']})
graph.commit(tx)  

ok


In [60]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
tx = graph.begin()
for index, row in gov_source.iterrows():
    tx.evaluate('''
    MERGE (s:government_host {IP: $SourceIP})
    MERGE (d:external_host {IP:$DestinationIP})
    CREATE (s)-[i:session]->(d)
    SET i.Bytes = toInteger($bytes),i.DestinationPort=$DestinationPort,i.Time=$time,i.Packets=$packets,i.soure_ip_intel=$soure_ip_intel,i.destination_ip_intel=$destination_ip_intel
    ''', parameters = {'SourceIP': row['source.ip'], 'DestinationIP': row['destination.ip'],'DestinationPort': row['destination.port'],'packets': row['network.total_packets'],'bytes': row['network.total_bytes'],'time': row['@timestamp'],'soure_ip_intel': row['soure.ip.intel'],'destination_ip_intel': row['destination.ip.intel']})
graph.commit(tx)  

ok


In [59]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
my_node = graph.run("MATCH (n) DETACH DELETE n")

ok
